In [ ]:
# import needed libraries
import time
from random import *
from matplotlib import animation, pyplot as plt
import numpy as np
import numba

In [ ]:
# gathering information for initializing the forest

GROUND = 0 # represents an empty ground
NON_BURNING_TREE = 1 # represents an area with a non burning tree
BURNING_TREE = 2 # represents an area with a burning tree
BORDERLINE = 3 # represents borderline around the forest

probTree = 0.8 # represents the probability that a tree initially ocupies the area
probBurning = 0.01 # represents the probability that a tree is burning
probImmune = 0.3 # represents the probability that a tree is immune to fire
probLightning = 0.001 # represents the probability that an area suffered a lightning strike

Non Parallel Simulation of the Spread

In [1]:
def initForestState(sizeOfForest):
    
    forestGrid = np.ones((sizeOfForest + 2, sizeOfForest + 2)) * BORDERLINE
    
    for i in range(1, sizeOfForest + 1):
        for j in range(1, sizeOfForest + 1):
            
            if random() < probBurning:
                # there is a burning tree in the area
                forestGrid[i, j] = BURNING_TREE
            elif random() < probTree:
                # the area contains a non burning tree
                forestGrid[i, j] = NON_BURNING_TREE
            else:
                # it's a fallow area
                forestGrid[i, j] = GROUND
                
    return forestGrid
    

In [2]:
# spread the fire with Moore neighborhood algorithm

def spreadTheFireWithMoore(sizeOfForest, forest):
    
    for i in range(1, sizeOfForest + 1):
        for j in range(1, sizeOfForest + 1):
            
            # if the area is a tree
            if forest[i,j] == NON_BURNING_TREE:
                
                # if the area is immune to fire the tree does not burn
                if random() < probImmune:
                    forest[i,j] = NON_BURNING_TREE
                else: 
                    
                    # if there's a burning tree is next to the area
                    # the moore neighborhoods (north, north-east, north-west, south, south-east, south-west, west, east)
                    # can tell us if there's a burning tree
                    if (forest[i - 1, j] == BURNING_TREE or forest[i + 1, j] == BURNING_TREE or 
                        forest[i, j - 1] == BURNING_TREE or forest[i, j + 1] == BURNING_TREE or 
                        forest[i - 1, j - 1] == BURNING_TREE or forest[i - 1, j + 1] == BURNING_TREE or 
                        forest[i + 1, j - 1] == BURNING_TREE or forest[i + 1, j + 1] == BURNING_TREE):
                        forest[i - 1, j] = BURNING_TREE # tree will burn
                        
                    # if the area suffered a lightning strike the tree burns
                    elif random() < probLightning: 
                        forest[i, j] = BURNING_TREE
                    else:
                        forest[i, j] = NON_BURNING_TREE # the tree doesn't burn
            
            # if the area is a burning tree, then the tree burns to the ground
            elif forest[i, j] == BURNING_TREE:
                forest[i, j] = GROUND
            
            # otherwise the area remains empty
            else:
                forest[i, j] = GROUND
            
    return forest

In [ ]:
def VisualizeForestFireSpread(sizeOfForest, noOfIteration):
        
    forest = initForestState(sizeOfForest)
        
    fig = plt.figure()
    ax = plt.axes(xlim=(0, sizeOfForest + 2), ylim=(0, sizeOfForest + 2))
    plt.title('Simulation of Forest Fire Spread With Size {0}'.format(sizeOfForest))
    plt.xlabel('x')
    plt.ylabel('y')
      
    # plot the forest
    plt.imshow(forest, cmap = 'binary')
        
    # plot the initial state of the forest
    plt.show()
        
    # plot the forest after each iteration
    for i in range(noOfIteration):
        forest = spreadTheFireWithMoore(sizeOfForest, forest)
        plt.imshow(forest, cmap = 'binary')
        plt.show()    

In [ ]:
startTime = time.time()

VisualizeForestFireSpread(100, 10)

timeElapsed = time.time() - startTime
print("Time elapsed for applying the spread on forest grid size 100: {0} seconds".format(timeElapsed))

In [ ]:
startTime = time.time()

VisualizeForestFireSpread(400, 10)

timeElapsed = time.time() - startTime
print("Time elapsed for applying the spread on forest grid size 400: {0} seconds".format(timeElapsed))

In [ ]:
startTime = time.time()

VisualizeForestFireSpread(800, 10)

timeElapsed = time.time() - startTime
print("Time elapsed for applying the spread on forest grid size 800: {0} seconds".format(timeElapsed))

In [ ]:
startTime = time.time()

VisualizeForestFireSpread(1000, 10)

timeElapsed = time.time() - startTime
print("Time elapsed for applying the spread on forest grid size 1000: {0} seconds".format(timeElapsed))

In [ ]:
startTime = time.time()

VisualizeForestFireSpread(1200, 10)

timeElapsed = time.time() - startTime
print("Time elapsed for applying the spread on forest grid size 1200: {0} seconds".format(timeElapsed))

In [ ]:
startTime = time.time()

VisualizeForestFireSpread(2000, 10)

timeElapsed = time.time() - startTime
print("Time elapsed for applying the spread on forest grid size 2000: {0} seconds".format(timeElapsed))

Parallelizing the Simulation of the Spread

In [ ]:
# Parallelizing the initForestState function

numba.jit(nopython=True, parallel=True)
def initForestState_Parallel(sizeOfForest):
    
    forestGrid = np.ones((sizeOfForest + 2, sizeOfForest + 2)) * BORDERLINE
    
    for i in numba.prange(1, sizeOfForest + 1):
        for j in numba.prange(1, sizeOfForest + 1):
            if random() < probBurning:
                forestGrid[i, j] = BURNING_TREE
            elif random() < probTree:
                forestGrid[i, j] = NON_BURNING_TREE
            else:
                forestGrid[i, j] = GROUND
                
    return forestGrid   
    

In [3]:
# Parallelizing the spread of the fire with Moore neighborhood algorithm

numba.jit(nopython=True, parallel=True)
def spreadTheFireWithMoore_Parallel(sizeOfForest, forestGrid):
    
    for i in numba.prange(1, sizeOfForest + 1):
        for j in numba.prange(1, sizeOfForest + 1):
            
            # if the area is a tree
            if forestGrid[i,j] == NON_BURNING_TREE:
                
                # if the area is immune to fire the tree does not burn
                if random() < probImmune:
                    forestGrid[i,j] = NON_BURNING_TREE
                else: 
                    
                    # if there's a burning tree is next to the area
                    if (forestGrid[i - 1, j] == BURNING_TREE or forestGrid[i + 1, j] == BURNING_TREE or 
                        forestGrid[i, j - 1] == BURNING_TREE or forestGrid[i, j + 1] == BURNING_TREE or 
                        forestGrid[i - 1, j - 1] == BURNING_TREE or forestGrid[i - 1, j + 1] == BURNING_TREE or 
                        forestGrid[i + 1, j - 1] == BURNING_TREE or forestGrid[i + 1, j + 1] == BURNING_TREE):
                        forestGrid[i - 1, j] = BURNING_TREE # tree will burn
                        
                    # if the area suffered a lightning strike the tree burns
                    elif random() < probLightning: 
                        forestGrid[i, j] = BURNING_TREE
                    else:
                        forestGrid[i, j] = NON_BURNING_TREE # the tree doesn't burn
            
            # if the area is a burning tree, then the tree burns to the ground
            elif forestGrid[i, j] == BURNING_TREE:
                forestGrid[i, j] = GROUND
            
            # otherwise the area remains empty
            else:
                forestGrid[i, j] = GROUND
            
    return forestGrid

In [ ]:
numba.jit(nopython=True, parallel=True)
def VisualizeForestFireSpread_Parallel(sizeOfForest, noOfIteration):
        
    forest = initForestState_Parallel(sizeOfForest)
        
    fig = plt.figure()
    ax = plt.axes(xlim=(0, sizeOfForest + 2), ylim=(0, sizeOfForest + 2))
    plt.title('Simulation of Forest Fire Spread With Size {0}'.format(sizeOfForest))
    plt.xlabel('x')
    plt.ylabel('y')
      
    # plot the forest
    plt.imshow(forest, cmap = 'binary')
        
    # plot the initial state of the forest
    plt.show()
        
    # plot the forest after each iteration
    for i in numba.prange(noOfIteration):
        forest = spreadTheFireWithMoore_Parallel(sizeOfForest, forest)
        plt.imshow(forest, cmap = 'binary')
        plt.show()

In [ ]:
startTime = time.time()

VisualizeForestFireSpread_Parallel(100, 10)

timeElapsed = time.time() - startTime
print("Time elapsed for applying the spread (Parallel) on forest grid size 100: {0} seconds".format(timeElapsed))

In [ ]:
startTime = time.time()

VisualizeForestFireSpread_Parallel(400, 10)

timeElapsed = time.time() - startTime
print("Time elapsed for applying the spread (Parallel) on forest grid size 400: {0} seconds".format(timeElapsed))

In [ ]:
startTime = time.time()

VisualizeForestFireSpread_Parallel(800, 10)

timeElapsed = time.time() - startTime
print("Time elapsed for applying the spread (Parallel) on forest grid size 800: {0} seconds".format(timeElapsed))

In [ ]:
startTime = time.time()

VisualizeForestFireSpread_Parallel(1000, 10)

timeElapsed = time.time() - startTime
print("Time elapsed for applying the spread (Parallel) on forest grid size 1000: {0} seconds".format(timeElapsed))

In [ ]:
startTime = time.time()

VisualizeForestFireSpread_Parallel(1200, 10)

timeElapsed = time.time() - startTime
print("Time elapsed for applying the spread (Parallel) on forest grid size 1200: {0} seconds".format(timeElapsed))

In [ ]:
startTime = time.time()

VisualizeForestFireSpread_Parallel(2000, 10)

timeElapsed = time.time() - startTime
print("Time elapsed for applying the spread (Parallel) on forest grid size 2000: {0} seconds".format(timeElapsed))